In [1]:
import pandas as pd
from passivetotal import analyzer
analyzer.init()
# set the date range we want the passivetotal analyzer to look at, see https://passivetotal.readthedocs.io/en/latest/analyzer.html#initialization for more
analyzer.set_date_range(start='2022-12-01', end='2023-01-31') 

In [2]:
%%bash
curl https://raw.githubusercontent.com/mitchellkrogza/Phishing.Database/master/phishing-domains-NEW-today.txt | head -n 20 > domains.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9846  100  9846    0     0  32690      0 --:--:-- --:--:-- --:--:-- 34426


In [3]:
# instead of using curl, you could simply start with a .txt file containing domains you retrieved from some other data source

with open('domains.txt', encoding='utf-8') as foo:
    f = foo.readlines()

output = pd.DataFrame()

for item in f:
    try:
        host = analyzer.Hostname(item.strip())
        resolutions = host.resolutions
        records = resolutions.only_a_records.as_df
        output = pd.concat([output, records])
    except:
        print("unable to return resolutions for " + item)

unable to return resolutions for 155.94.197.69

unable to return resolutions for 18.210.25.103

unable to return resolutions for 23.94.92.5

unable to return resolutions for 31.220.58.83

unable to return resolutions for 3.229.240.77

unable to return resolutions for 34.204.124.168

unable to return resolutions for 34.237.197.30

unable to return resolutions for 34.72.129.252

unable to return resolutions for 44.195.104.3



In [4]:
output

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
0,0000mscautorizationclientid.com,A,104.21.81.185,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
1,0000mscautorizationclientid.com,A,188.114.96.14,ip,2023-02-01 01:35:06,2023-01-16 02:47:48,2023-01-16 02:48:07,0,[riskiq]
2,0000mscautorizationclientid.com,A,188.114.97.2,ip,2023-02-01 01:35:06,2023-01-17 01:07:22,2023-01-17 01:07:22,0,[riskiq]
3,0000mscautorizationclientid.com,A,188.114.96.2,ip,2023-02-01 01:35:06,2023-01-17 01:07:22,2023-01-17 01:07:22,0,[riskiq]
4,0000mscautorizationclientid.com,A,188.114.97.0,ip,2023-02-01 01:35:06,2023-01-16 02:49:33,2023-01-16 02:49:33,0,[riskiq]
5,0000mscautorizationclientid.com,A,188.114.96.0,ip,2023-02-01 01:35:06,2023-01-16 02:49:33,2023-01-16 02:49:33,0,[riskiq]
6,0000mscautorizationclientid.com,A,172.67.163.98,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
7,0000mscautorizationclientid.com,A,188.114.97.14,ip,2023-02-01 01:35:06,2023-01-16 02:47:48,2023-01-16 02:48:07,0,[riskiq]
0,0ec50dc.wcomhost.com,A,206.188.193.211,ip,2023-02-01 01:35:07,2023-01-16 11:46:02,2023-01-18 10:16:56,1,[riskiq]
1,0ec50dc.wcomhost.com,A,208.91.197.27,ip,2023-02-01 01:35:07,2023-01-18 09:48:17,2023-01-31 17:35:07,13,"[riskiq, pingly]"


In [5]:
output = output[output['firstseen'] > '2023-01-01']

In [6]:
output = output[output['duration'] > 0]

In [7]:
output

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
0,0000mscautorizationclientid.com,A,104.21.81.185,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
6,0000mscautorizationclientid.com,A,172.67.163.98,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
0,0ec50dc.wcomhost.com,A,206.188.193.211,ip,2023-02-01 01:35:07,2023-01-16 11:46:02,2023-01-18 10:16:56,1,[riskiq]
1,0ec50dc.wcomhost.com,A,208.91.197.27,ip,2023-02-01 01:35:07,2023-01-18 09:48:17,2023-01-31 17:35:07,13,"[riskiq, pingly]"
0,19-0-1-vetrge5tr0-9g-0wrhyf0-9qewjf-0hwf9-hgh....,A,159.138.16.166,ip,2023-02-01 01:35:09,2023-01-12 09:08:45,2023-01-31 17:35:09,19,"[riskiq, pingly]"
0,22494-4867.s1.webspace.re,A,45.88.108.231,ip,2023-02-01 01:35:10,2023-01-17 01:04:56,2023-01-31 17:35:09,14,"[riskiq, pingly]"
2,5ff6r-paaaa-aaaap-aax3q-cai.raw.ic0.app,A,139.178.83.182,ip,2023-02-01 01:35:11,2023-01-16 02:48:42,2023-01-31 17:35:10,15,"[riskiq, pingly]"
0,7fe63d7540358a.lhr.life,A,3.234.18.192,ip,2023-02-01 01:35:13,2023-01-16 15:35:42,2023-01-31 17:35:12,15,"[riskiq, pingly]"
1,7fe63d7540358a.lhr.life,A,3.208.46.244,ip,2023-02-01 01:35:13,2023-01-16 14:51:17,2023-01-31 17:35:12,15,"[riskiq, pingly]"
2,7fe63d7540358a.lhr.life,A,54.172.225.3,ip,2023-02-01 01:35:13,2023-01-16 15:02:20,2023-01-31 17:35:12,15,"[riskiq, pingly]"


In [8]:
# display pDNS resolutions for the domains we retrieved, sorting by first seen timestamp

output.sort_values(by=['firstseen'], ascending=False)

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
1,0ec50dc.wcomhost.com,A,208.91.197.27,ip,2023-02-01 01:35:07,2023-01-18 09:48:17,2023-01-31 17:35:07,13,"[riskiq, pingly]"
0,22494-4867.s1.webspace.re,A,45.88.108.231,ip,2023-02-01 01:35:10,2023-01-17 01:04:56,2023-01-31 17:35:09,14,"[riskiq, pingly]"
0,7fe63d7540358a.lhr.life,A,3.234.18.192,ip,2023-02-01 01:35:13,2023-01-16 15:35:42,2023-01-31 17:35:12,15,"[riskiq, pingly]"
2,7fe63d7540358a.lhr.life,A,54.172.225.3,ip,2023-02-01 01:35:13,2023-01-16 15:02:20,2023-01-31 17:35:12,15,"[riskiq, pingly]"
1,7fe63d7540358a.lhr.life,A,3.208.46.244,ip,2023-02-01 01:35:13,2023-01-16 14:51:17,2023-01-31 17:35:12,15,"[riskiq, pingly]"
0,0ec50dc.wcomhost.com,A,206.188.193.211,ip,2023-02-01 01:35:07,2023-01-16 11:46:02,2023-01-18 10:16:56,1,[riskiq]
0,9646464544.hyperphp.com,A,185.27.134.179,ip,2023-02-01 01:35:14,2023-01-16 08:05:28,2023-01-31 17:35:13,15,"[riskiq, pingly, mnemonic]"
0,0000mscautorizationclientid.com,A,104.21.81.185,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
6,0000mscautorizationclientid.com,A,172.67.163.98,ip,2023-02-01 01:35:06,2023-01-16 02:54:31,2023-01-17 07:11:26,1,[riskiq]
2,5ff6r-paaaa-aaaap-aax3q-cai.raw.ic0.app,A,139.178.83.182,ip,2023-02-01 01:35:11,2023-01-16 02:48:42,2023-01-31 17:35:10,15,"[riskiq, pingly]"


In [9]:
# do a reverse lookup on the IP addresses in the above dataframe to find A records for all domains resolving to these IP addresses

reverse_df = pd.DataFrame()

for ip in output["resolve"]:
    reverse_df = pd.concat([reverse_df,analyzer.IPAddress(ip).resolutions.only_a_records.as_df])

In [10]:
reverse_df

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
0,104.21.81.185,A,www.lingermt.com.cdn.cloudflare.net,domain,2023-02-01 01:34:32,2022-08-24 08:16:49,2022-12-31 04:33:25,128,[riskiq]
1,104.21.81.185,A,www.abogadodefamilia.co.cdn.cloudflare.net,domain,2023-02-01 01:34:32,2022-08-13 04:30:13,2023-01-17 11:21:13,157,[riskiq]
2,104.21.81.185,A,vivigon.co,domain,2023-02-01 01:35:15,2022-07-09 22:29:28,2023-01-23 11:24:52,197,[riskiq]
3,104.21.81.185,A,www.lanzaequip.es,domain,2023-02-01 01:35:15,2022-06-14 07:12:17,2022-12-19 02:22:22,187,[riskiq]
4,104.21.81.185,A,argopan.com,domain,2023-02-01 01:34:32,2022-11-02 16:59:44,2023-01-17 05:51:10,75,[riskiq]
...,...,...,...,...,...,...,...,...,...
364,185.27.134.179,A,ifsowo.000space.com,domain,2023-02-01 01:34:27,2022-11-10 19:23:01,2023-01-30 11:46:07,80,[riskiq]
365,185.27.134.179,A,www.seulink.ga,domain,2023-02-01 01:35:33,2023-01-09 16:44:16,2023-01-09 16:44:16,0,[riskiq]
366,185.27.134.179,A,wipeirabedqe.000space.com,domain,2023-02-01 01:34:27,2022-11-12 04:28:11,2023-01-21 19:54:21,70,[riskiq]
367,185.27.134.179,A,thegamer.ga,domain,2023-02-01 01:35:33,2022-08-30 19:15:49,2023-01-24 13:33:19,146,[riskiq]


In [11]:
reverse_filtered = reverse_df[(reverse_df['lastseen'] > '2023-01-15') & (reverse_df['firstseen'] > '2023-01-15') ]
# because pandas automatically recognizes that these colums are datetimes, you can filter easily

In [12]:
reverse_filtered[reverse_filtered['duration'] > 0]

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
259,104.21.81.185,A,yeetbox.au,domain,2023-02-01 01:35:15,2023-01-24 18:23:11,2023-01-26 16:41:44,1,[riskiq]
268,104.21.81.185,A,www.the-landrovers.com,domain,2023-02-01 01:34:32,2023-01-21 15:41:50,2023-01-26 08:37:03,4,[riskiq]
85,172.67.163.98,A,*.ambries.bar,wildcard,2023-02-01 01:35:17,2023-01-27 20:40:22,2023-01-29 17:36:10,1,[riskiq]
107,172.67.163.98,A,yeetbox.au,domain,2023-02-01 01:35:17,2023-01-24 18:23:11,2023-01-26 16:41:44,1,[riskiq]
201,172.67.163.98,A,www.the-landrovers.com,domain,2023-02-01 01:35:17,2023-01-17 21:57:52,2023-01-26 01:36:42,8,[riskiq]
...,...,...,...,...,...,...,...,...,...
88,185.27.134.179,A,amazonfast.ml,domain,2023-02-01 01:34:27,2023-01-22 17:14:13,2023-01-30 12:53:03,7,[riskiq]
91,185.27.134.179,A,learnfastest.cf,domain,2023-02-01 01:35:33,2023-01-23 09:45:26,2023-01-25 19:22:56,2,[riskiq]
176,185.27.134.179,A,notateknoloji.cf,domain,2023-02-01 01:35:33,2023-01-18 18:26:28,2023-01-21 23:36:46,3,[riskiq]
223,185.27.134.179,A,ardacamtepe.cf,domain,2023-02-01 01:35:33,2023-01-17 15:25:00,2023-01-31 03:42:24,13,[riskiq]


In [13]:
# analyzer.Hostname('crowdstrike.com').whois returns current whois results for a given domain.
# we call this, again with the as_df method, and store the results in a dataframe to compare WHOIS data

whois_df = pd.DataFrame()

for domain in output["query"].drop_duplicates():
    try:
        whois_df = pd.concat([whois_df,analyzer.Hostname(domain.strip()).whois.as_df])
    except:
        pass

In [14]:
whois_df

,query,organization,name,telephone,email,registrant_org,registrant_name,registrant_phone,registrant_email,registrar,server,age,date_registered,date_updated,date_loaded,nameservers,date_expires
0,0000mscautorizationclientid.com,"Domains By Proxy, LLC",Registration Private,+1.4806242599,abuse@godaddy.com,"Domains By Proxy, LLC",Registration Private,+1.4806242599,select contact domain holder link at https://w...,"GoDaddy.com, LLC",rdap.godaddy.com,15,2023-01-15 20:50:12-08:00,2023-01-15 20:50:12-08:00,2023-01-16 09:52:57.165000-08:00,"[aurora.ns.cloudflare.com, lou.ns.cloudflare.com]",2024-01-15 20:50:12-08:00
0,wcomhost.com,Network Solutions LLC,"LLC, networksolutions",+1.7036684900,domains@web.com,Network Solutions LLC,"LLC, networksolutions",+1.7036684900,domains@web.com,"Network Solutions, LLC",rdap.networksolutions.com,4024,2012-01-25 12:25:03-08:00,2022-01-25 15:05:12-08:00,2023-01-22 14:44:11.857000-08:00,"[ns51.worldnic.com, ns52.worldnic.com]",2032-01-25 12:25:03-08:00
0,myhuaweicloud.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,N/A,supervision@xinnet.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,,supervision@xinnet.com,Xin Net Technology Corporation,whois.paycenter.com.cn,1931,2017-10-17 19:27:23-07:00,2022-02-21 06:41:09-08:00,2023-01-23 19:52:30.946000-08:00,"[vip3.alidns.com, vip4.alidns.com]",2030-10-17 19:27:23-07:00
0,webspace.re,N/A,N/A,N/A,,,,,,None,whois.nic.fr,None,None,None,2022-12-26 11:53:14.756000-08:00,"[ivy.ns.cloudflare.com, paul.ns.cloudflare.com]",None
0,ic0.app,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,redacted for privacy,registrar-abuse@google.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,redacted for privacy,,Google LLC.,www.registry.google,985,2020-05-21 11:50:35.406000-07:00,2022-11-07 04:24:07.409000-08:00,2023-01-31 02:48:59.745000-08:00,"[clay.ns.cloudflare.com, karina.ns.cloudflare....",2023-05-21 11:50:35.406000-07:00
0,lhr.life,Identity Protection Service,On behalf of lhr.life owner,+44.1483307527,79c735ec-5a19-4b00-974a-ebc9d771a33e@identity-...,Identity Protection Service,On behalf of lhr.life owner,+44.1483307527,79c735ec-5a19-4b00-974a-ebc9d771a33e@identity-...,"Amazon Registrar, Inc.",rdap.registrar.amazon.com,438,2021-11-19 00:56:10-08:00,2022-10-15 16:05:39-07:00,2023-01-24 05:26:39.801000-08:00,"[ns-778.awsdns-33.net, ns-471.awsdns-58.com, n...",2023-11-19 00:56:10-08:00
0,hyperphp.com,Redacted for Privacy Purposes,Redacted for Privacy Purposes,redacted for privacy purposes,support@namecheap.com,Redacted for Privacy Purposes,Redacted for Privacy Purposes,redacted for privacy purposes,select contact domain holder link at https://w...,NAMECHEAP INC,rdap.namecheap.com,5949,2006-10-18 07:56:27-07:00,2022-09-18 00:03:52-07:00,2023-01-23 06:01:13.170000-08:00,"[ns1.byet.org, ns2.byet.org, ns3.byet.org, ns4...",2023-10-18 07:56:27-07:00


In [15]:
# the pandas value_counts method returns a series in descending order of each unique value by frequency https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
# below are just a few examples of things you can do to look for most prevalent values

whois_df["organization"].value_counts()

REDACTED FOR PRIVACY             2
Domains By Proxy, LLC            1
Network Solutions LLC            1
N/A                              1
Identity Protection Service      1
Redacted for Privacy Purposes    1
Name: organization, dtype: int64

In [16]:
whois_df["registrar"].value_counts()

GoDaddy.com, LLC                  1
Network Solutions, LLC            1
Xin Net Technology Corporation    1
Google LLC.                       1
Amazon Registrar, Inc.            1
NAMECHEAP INC                     1
Name: registrar, dtype: int64

In [17]:
whois_df["nameservers"].value_counts()

[aurora.ns.cloudflare.com, lou.ns.cloudflare.com]                                               1
[ns51.worldnic.com, ns52.worldnic.com]                                                          1
[vip3.alidns.com, vip4.alidns.com]                                                              1
[ivy.ns.cloudflare.com, paul.ns.cloudflare.com]                                                 1
[clay.ns.cloudflare.com, karina.ns.cloudflare.com]                                              1
[ns-778.awsdns-33.net, ns-471.awsdns-58.com, ns-1349.awsdns-40.org, ns-2045.awsdns-63.co.uk]    1
[ns1.byet.org, ns2.byet.org, ns3.byet.org, ns4.byet.org, ns5.byet.org]                          1
Name: nameservers, dtype: int64

In [18]:
# get just SOA records

soa_df = pd.DataFrame()

for domain in output["query"].drop_duplicates():
    try:
        res_df = analyzer.Hostname(domain.strip()).resolutions.as_df
        # there is no method akin to only_a_records, hence the filtering by recordtype
        new_soa = res_df[(res_df["recordtype"] == 'SOA')]
        soa_df = pd.concat([soa_df,new_soa])
    except:
        print("error in requesting SOA record for " + domain.strip())

In [19]:
soa_df

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
8,0000mscautorizationclientid.com,SOA,aurora.ns.cloudflare.com,domain,2023-02-01 01:35:06,2023-01-16 02:49:44,2023-01-16 23:52:43,0,[riskiq]
17,0000mscautorizationclientid.com,SOA,dns@cloudflare.com,email,2023-02-01 01:35:06,2023-01-16 02:49:44,2023-01-16 23:52:43,0,[riskiq]


In [20]:
# list the most common SOA email addresses
soa_df[(soa_df['resolvetype'] == 'email')]['resolve'].value_counts()

dns@cloudflare.com    1
Name: resolve, dtype: int64

In [21]:
mx_df = pd.DataFrame()

for domain in output["query"].drop_duplicates():
    try:
        res_df = analyzer.Hostname(domain.strip()).resolutions.as_df
        # there is no method akin to only_a_records, hence the filtering by recordtype
        new_mx = res_df[(res_df["recordtype"] == 'MX')]
        mx_df = pd.concat([mx_df,new_mx])
    except:
        print("error in obtaining mx record for " + domain.strip())

In [22]:
mx_df

,query,recordtype,resolve,resolvetype,collected,firstseen,lastseen,duration,sources
1,9646464544.hyperphp.com,MX,mx.byetcluster.com,domain,2023-02-01 01:35:13,2023-01-16 09:03:04,2023-01-31 17:35:13,15,"[riskiq, pingly]"


In [23]:
mx_df['resolve'].value_counts()

mx.byetcluster.com    1
Name: resolve, dtype: int64

In [24]:
whois_df.to_csv('whois_df.csv')

In [25]:
mx_df.to_csv('mx_df.csv')

In [26]:
soa_df.to_csv('soa_df.csv')

In [27]:
ip_df = pd.DataFrame()

for ip in output['resolve'].drop_duplicates():
    IP = analyzer.IPAddress(ip)
    ip_info_df = IP.summary.as_df
    ip_df = pd.concat([ip_df, ip_info_df])

In [28]:
ip_df

,host,total,articles,certificates,malware_hashes,projects,resolutions,netblock,os,asn
0,104.21.81.185,1953,0,329,0,0,1612,104.21.80.0/20,n/a,AS13335 - CLOUDFLARENET
0,172.67.163.98,2112,0,394,0,0,1706,172.67.160.0/20,n/a,AS13335 - CLOUDFLARENET
0,206.188.193.211,2053,0,60,0,0,1987,206.188.193.0/24,Gentoo,AS19871 - NETWORK-SOLUTIONS-HOSTING
0,208.91.197.27,191831,0,189828,0,1,1999,208.91.197.0/24,Debian,AS40034 - CONFLUENCE-NETWORK-INC
0,159.138.16.166,166,0,3,0,0,160,159.138.16.0/22,n/a,AS136907 - HWCLOUDS-AS-AP
0,45.88.108.231,2709,0,699,0,0,1994,45.88.108.0/24,Debian,AS44486 - SYNLINQ
0,139.178.83.182,1012,0,7,0,0,1003,139.178.80.0/21,n/a,AS54825 - PACKET
0,3.234.18.192,1109,0,6,0,0,1100,3.224.0.0/12,n/a,AS14618 - AMAZON-AES
0,3.208.46.244,1111,0,9,0,0,1099,3.208.0.0/12,n/a,AS14618 - AMAZON-AES
0,54.172.225.3,1116,0,6,0,0,1107,54.172.0.0/15,n/a,AS14618 - AMAZON-AES


In [29]:
ip_df['asn'].value_counts()

AS14618 - AMAZON-AES                   3
AS13335 - CLOUDFLARENET                2
AS19871 - NETWORK-SOLUTIONS-HOSTING    1
AS40034 - CONFLUENCE-NETWORK-INC       1
AS136907 - HWCLOUDS-AS-AP              1
AS44486 - SYNLINQ                      1
AS54825 - PACKET                       1
AS34119 - WILDCARD-AS                  1
Name: asn, dtype: int64

In [30]:
ip_df['netblock'].value_counts()

104.21.80.0/20      1
172.67.160.0/20     1
206.188.193.0/24    1
208.91.197.0/24     1
159.138.16.0/22     1
45.88.108.0/24      1
139.178.80.0/21     1
3.224.0.0/12        1
3.208.0.0/12        1
54.172.0.0/15       1
185.27.132.0/22     1
Name: netblock, dtype: int64

In [31]:
ip_df.to_csv('ip_df.csv')